## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Oct. 20th Oct. 27th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-22-2024,Condo/Co-op,3100 N Ocean Dr Unit 1606 H,West Palm Beach,FL,33404.0,1500000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,F10467804,N,Y,26.787798,-80.034574


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
39,PAST SALE,2024-10-25,Condo/Co-op,9431 Boca Cove Cir #1003,Boca Raton,FL,33428.0,250000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/9431-Boca...,Beaches MLS,RX-11010452,N,Y,26.328202,-80.193392
63,PAST SALE,2024-10-25,Condo/Co-op,90 Mansfield C,Boca Raton,FL,33434.0,75000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/90-Mansfi...,Beaches MLS,RX-10979447,N,Y,26.391221,-80.174736
25,PAST SALE,2024-10-25,Condo/Co-op,3661 Via Poinciana #307,Lake Worth,FL,33467.0,115000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/3661-Via-...,Beaches MLS,RX-11016998,N,Y,26.622586,-80.153979
42,PAST SALE,2024-10-25,Condo/Co-op,12565 Imperial Isle Dr #206,Boynton Beach,FL,33437.0,314000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/12565-...,Beaches MLS,RX-11008831,N,Y,26.489217,-80.133683
60,PAST SALE,2024-10-25,Condo/Co-op,20 Eastgate Dr Unit C,Boynton Beach,FL,33436.0,195000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/20-Eas...,Beaches MLS,RX-10990619,N,Y,26.487921,-80.096275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,PAST SALE,2024-10-21,Condo/Co-op,5530 N Ocean Blvd #102,Ocean Ridge,FL,33435.0,375000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Ocean-Ridge/5530-N-O...,Beaches MLS,RX-11002824,N,Y,26.512029,-80.055467
66,PAST SALE,2024-10-21,Condo/Co-op,1101 Green Pine Blvd Unit A1,West Palm Beach,FL,33409.0,200000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1101...,Beaches MLS,RX-10986703,N,Y,26.728659,-80.104066
65,PAST SALE,2024-10-21,Condo/Co-op,1200 Marine Way #903,North Palm Beach,FL,33408.0,292000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/120...,Beaches MLS,RX-10987088,N,Y,26.834929,-80.059224
27,PAST SALE,2024-10-21,Condo/Co-op,67 Kent E,West Palm Beach,FL,33417.0,140000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/67-K...,Beaches MLS,RX-11005235,N,Y,26.715355,-80.131526


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

80


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$347,880


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$256


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$27,830,387


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach
Input Previous Week Condo Sales Total (number units sold):
78
Input Previous Week Condo Average Sales Price:
553339
Input Previous Week Condo Average PSF:
328
Input Previous Week Condo Sales total (ex: 198_000_000)
45_900_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-23,Condo/Co-op,200 Arkona Ct #1104,West Palm Beach,FL,33401.0,6500000.0,3.0,3.5,1515 Flagler Condo,3094.0,NaN,2023.0,NaN,2101.0,5330.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11015480,N,Y,26.699821,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Michael Costello,Compass Florida LLC (PB),Chris Deitz,Compass Florida LLC (PB),Michael Costello,Compass Florida LLC (PB),None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-23,Condo/Co-op,200 Arkona Ct #1104,West Palm Beach,FL,33401.0,6500000.0,3.0,3.5,1515 Flagler Condo,3094.0,NaN,2023.0,NaN,2101.0,5330.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11015480,N,Y,26.699821,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Michael Costello,Compass Florida LLC (PB),Chris Deitz,Compass Florida LLC (PB),Michael Costello,Compass Florida LLC (PB),None,None,1,orange
1,PAST SALE,2024-10-22,Condo/Co-op,3100 N Ocean Dr Unit 1606 H,West Palm Beach,FL,33404.0,1500000.0,2.0,2.5,Amrit Ocean Resort & Resi,1600.0,NaN,2023.0,NaN,938.0,2320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,F10467804,N,Y,26.787798,-80.034574,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Kari Lydon,Compass Florida LLC,None,None,Ryan Kartheiser,"Miami Condo Advisors, LLC",None,None,2,blue
2,PAST SALE,2024-10-25,Condo/Co-op,500 S Ocean Blvd #604,Boca Raton,FL,33432.0,1300000.0,2.0,2.0,Chalfonte Condo,1446.0,NaN,1974.0,NaN,899.0,1569.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Beaches MLS,RX-11013796,N,Y,26.345068,-80.071816,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,3,blue
3,PAST SALE,2024-10-22,Condo/Co-op,2103 S Ocean Blvd Unit 3-C,Delray Beach,FL,33483.0,1275000.0,2.0,2.0,Bermuda High South,1254.0,NaN,1968.0,NaN,1017.0,1777.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2103-S-...,Beaches MLS,RX-10976860,N,Y,26.434986,-80.062616,https://www.redfin.com/FL/Delray-Beach/2103-S-...,Karen Anderson,Champagne & Parisi Real Estate,None,None,Natalia Reid,One Sotheby's International Realty,None,None,4,blue
4,PAST SALE,2024-10-25,Condo/Co-op,123 Tresana Blvd #57,Jupiter,FL,33478.0,957500.0,3.0,3.0,Jupiter Country Club Condo II,2665.0,NaN,2015.0,NaN,359.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/123-Tresana-...,Beaches MLS,RX-10994966,N,Y,26.926523,-80.157293,https://www.redfin.com/FL/Jupiter/123-Tresana-...,Mary Ellen Kavanewsky,"NV Realty Group, LLC",None,None,Anne Bosso,"Robert Bosso Rlty Ser., Inc",None,None,5,blue
5,PAST SALE,2024-10-25,Condo/Co-op,99 SE Mizner Blvd #738,Boca Raton,FL,33432.0,535000.0,1.0,1.0,Palmetto Place Condo,890.0,NaN,2003.0,NaN,601.0,868.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/99-SE-Miz...,Beaches MLS,RX-10996459,N,Y,26.348949,-80.083947,https://www.redfin.com/FL/Boca-Raton/99-SE-Miz...,Andrea Morrone,Keller Williams Realty Boca Raton,None,None,Nikki Hirsch,"United Realty Group, Inc",Stacey Swaye,"United Realty Group, Inc",6,blue
6,PAST SALE,2024-10-22,Condo/Co-op,1605 S Us Highway 1 Unit 10e,Jupiter,FL,33477.0,525000.0,2.0,2.0,Sea Lofters,1160.0,NaN,1981.0,NaN,453.0,769.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1605-S-US-Hi...,Beaches MLS,RX-11017589,N,Y,26.921071,-80.071591,https://www.redfin.com/FL/Jupiter/1605-S-US-Hi...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Bradley Ball,Frankel Ball Realty LLC,None,None,7,blue
7,PAST SALE,2024-10-25,Condo/Co-op,224 Bay Colony Dr N,Juno Beach,FL,33408.0,505000.0,2.0,2.0,North Bay Colony Condo,1206.0,NaN,1995.0,NaN,419.0,825.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/224-Bay-C...,Beaches MLS,RX-10998857,N,Y,26.861207,-80.064643,https://www.redfin.com/FL/Juno-Beach/224-Bay-C...,Helen McCulloch,Your Home Sold Guaranteed Realty Services,None,None,John Rupp,Premier Brokers International,None,None,8,blue
8,PAST SALE,2024-10-22,Condo/Co-op,17257 Boca Club Blvd #4,

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-23,Condo/Co-op,200 Arkona Ct #1104,West Palm Beach,FL,33401.0,6500000.0,3.0,3.5,1515 Flagler Condo,3094.0,NaN,2023.0,NaN,2101.0,5330.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11015480,N,Y,26.699821,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Michael Costello,Compass Florida LLC (PB),Chris Deitz,Compass Florida LLC (PB),Michael Costello,Compass Florida LLC (PB),None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_10282024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'1515 Flagler Condo at 200 Arkona Ct #1104 in West Palm Beach'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-23,Condo/Co-op,200 Arkona Ct #1104,West Palm Beach,FL,33401.0,6500000.0,3.0,3.5,1515 Flagler Condo,3094.0,NaN,2023.0,NaN,2101.0,5330.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11015480,N,Y,26.699821,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Michael Costello,Compass Florida LLC (PB),Chris Deitz,Compass Florida LLC (PB),Michael Costello,Compass Florida LLC (PB),None,None,1,orange,1515 Flagler Condo at 200 Arkona Ct #1104 in W...
1,PAST SALE,2024-10-22,Condo/Co-op,3100 N Ocean Dr Unit 1606 H,West Palm Beach,FL,33404.0,1500000.0,2.0,2.5,Amrit Ocean Resort & Resi,1600.0,NaN,2023.0,NaN,938.0,2320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,F10467804,N,Y,26.787798,-80.034574,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Kari Lydon,Compass Florida LLC,None,None,Ryan Kartheiser,"Miami Condo Advisors, LLC",None,None,2,blue,Amrit Ocean Resort & Resi at 3100 N Ocean Dr U...
2,PAST SALE,2024-10-25,Condo/Co-op,500 S Ocean Blvd #604,Boca Raton,FL,33432.0,1300000.0,2.0,2.0,Chalfonte Condo,1446.0,NaN,1974.0,NaN,899.0,1569.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Beaches MLS,RX-11013796,N,Y,26.345068,-80.071816,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,3,blue,Chalfonte Condo at 500 S Ocean Blvd #604 in Bo...
3,PAST SALE,2024-10-22,Condo/Co-op,2103 S Ocean Blvd Unit 3-C,Delray Beach,FL,33483.0,1275000.0,2.0,2.0,Bermuda High South,1254.0,NaN,1968.0,NaN,1017.0,1777.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2103-S-...,Beaches MLS,RX-10976860,N,Y,26.434986,-80.062616,https://www.redfin.com/FL/Delray-Beach/2103-S-...,Karen Anderson,Champagne & Parisi Real Estate,None,None,Natalia Reid,One Sotheby's International Realty,None,None,4,blue,Bermuda High South at 2103 S Ocean Blvd Unit 3...
4,PAST SALE,2024-10-25,Condo/Co-op,123 Tresana Blvd #57,Jupiter,FL,33478.0,957500.0,3.0,3.0,Jupiter Country Club Condo II,2665.0,NaN,2015.0,NaN,359.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/123-Tresana-...,Beaches MLS,RX-10994966,N,Y,26.926523,-80.157293,https://www.redfin.com/FL/Jupiter/123-Tresana-...,Mary Ellen Kavanewsky,"NV Realty Group, LLC",None,None,Anne Bosso,"Robert Bosso Rlty Ser., Inc",None,None,5,blue,Jupiter Country Club Condo II at 123 Tresana B...
5,PAST SALE,2024-10-25,Condo/Co-op,99 SE Mizner Blvd #738,Boca Raton,FL,33432.0,535000.0,1.0,1.0,Palmetto Place Condo,890.0,NaN,2003.0,NaN,601.0,868.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/99-SE-Miz...,Beaches MLS,RX-10996459,N,Y,26.348949,-80.083947,https://www.redfin.com/FL/Boca-Raton/99-SE-Miz...,Andrea Morrone,Keller Williams Realty Boca Raton,None,None,Nikki Hirsch,"United Realty Group, Inc",Stacey Swaye,"United Realty Group, Inc",6,blue,Palmetto Place Condo at 99 SE Mizner Blvd #738...
6,PAST SALE,2024-10-22,Condo/Co-op,1605 S Us Highway 1 Unit 10e,Jupiter,FL,33477.0,525000.0,2.0,2.0,Sea Lofters,1160.0,NaN,1981.0,NaN,453.0,769.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1605-S-US-Hi...,Beaches MLS,RX-11017589,N,Y,26.921071,-80.071591,https://www.redfin.com/FL/Jupiter/1605-S-US-Hi...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Bradley Ball,Frankel Ball Realty LLC,None,None,7,blue,Sea Lofters at 1605 S Us Highway 1 Unit 10e in...
7,PAST SALE,2024-10-25,Condo/Co-op,224 Bay Colony Dr N,Juno Beach,FL,33408.0,505000.0,2.0,2.0,North Bay Colony Condo,1206.0,NaN,1995.0,Na

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 1515 Flagler Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $500,000M to $6,500,000M
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, West Palm Beach, Delray Beach, Jupiter, Boca Raton, North Palm Beach, Juno Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 80 condo sales totaling $27,830,387 million from Oct. 20th to Oct. 27th. The previous week, brokers closed 78 condo sales totaling $45,900,000.

Last week’s units sold for an average of $347,880, lower than the $553,339 average price from the previous week.

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-23,Condo/Co-op,200 Arkona Ct #1104,West Palm Beach,FL,33401.0,6500000.0,3.0,3.5,1515 Flagler Condo,3094.0,NaN,2023.0,NaN,2101.0,5330.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11015480,N,Y,26.699821,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Michael Costello,Compass Florida LLC (PB),Chris Deitz,Compass Florida LLC (PB),Michael Costello,Compass Florida LLC (PB),None,None,1,orange,1515 Flagler Condo at 200 Arkona Ct #1104 in W...
1,PAST SALE,2024-10-22,Condo/Co-op,3100 N Ocean Dr Unit 1606 H,West Palm Beach,FL,33404.0,1500000.0,2.0,2.5,Amrit Ocean Resort & Resi,1600.0,NaN,2023.0,NaN,938.0,2320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,F10467804,N,Y,26.787798,-80.034574,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Kari Lydon,Compass Florida LLC,None,None,Ryan Kartheiser,"Miami Condo Advisors, LLC",None,None,2,blue,Amrit Ocean Resort & Resi at 3100 N Ocean Dr U...
2,PAST SALE,2024-10-25,Condo/Co-op,500 S Ocean Blvd #604,Boca Raton,FL,33432.0,1300000.0,2.0,2.0,Chalfonte Condo,1446.0,NaN,1974.0,NaN,899.0,1569.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Beaches MLS,RX-11013796,N,Y,26.345068,-80.071816,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,3,blue,Chalfonte Condo at 500 S Ocean Blvd #604 in Bo...
3,PAST SALE,2024-10-22,Condo/Co-op,2103 S Ocean Blvd Unit 3-C,Delray Beach,FL,33483.0,1275000.0,2.0,2.0,Bermuda High South,1254.0,NaN,1968.0,NaN,1017.0,1777.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2103-S-...,Beaches MLS,RX-10976860,N,Y,26.434986,-80.062616,https://www.redfin.com/FL/Delray-Beach/2103-S-...,Karen Anderson,Champagne & Parisi Real Estate,None,None,Natalia Reid,One Sotheby's International Realty,None,None,4,blue,Bermuda High South at 2103 S Ocean Blvd Unit 3...
4,PAST SALE,2024-10-25,Condo/Co-op,123 Tresana Blvd #57,Jupiter,FL,33478.0,957500.0,3.0,3.0,Jupiter Country Club Condo II,2665.0,NaN,2015.0,NaN,359.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/123-Tresana-...,Beaches MLS,RX-10994966,N,Y,26.926523,-80.157293,https://www.redfin.com/FL/Jupiter/123-Tresana-...,Mary Ellen Kavanewsky,"NV Realty Group, LLC",None,None,Anne Bosso,"Robert Bosso Rlty Ser., Inc",None,None,5,blue,Jupiter Country Club Condo II at 123 Tresana B...
5,PAST SALE,2024-10-25,Condo/Co-op,99 SE Mizner Blvd #738,Boca Raton,FL,33432.0,535000.0,1.0,1.0,Palmetto Place Condo,890.0,NaN,2003.0,NaN,601.0,868.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/99-SE-Miz...,Beaches MLS,RX-10996459,N,Y,26.348949,-80.083947,https://www.redfin.com/FL/Boca-Raton/99-SE-Miz...,Andrea Morrone,Keller Williams Realty Boca Raton,None,None,Nikki Hirsch,"United Realty Group, Inc",Stacey Swaye,"United Realty Group, Inc",6,blue,Palmetto Place Condo at 99 SE Mizner Blvd #738...
6,PAST SALE,2024-10-22,Condo/Co-op,1605 S Us Highway 1 Unit 10e,Jupiter,FL,33477.0,525000.0,2.0,2.0,Sea Lofters,1160.0,NaN,1981.0,NaN,453.0,769.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1605-S-US-Hi...,Beaches MLS,RX-11017589,N,Y,26.921071,-80.071591,https://www.redfin.com/FL/Jupiter/1605-S-US-Hi...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Bradley Ball,Frankel Ball Realty LLC,None,None,7,blue,Sea Lofters at 1605 S Us Highway 1 Unit 10e in...
7,PAST SALE,2024-10-25,Condo/Co-op,224 Bay Colony Dr N,Juno Beach,FL,33408.0,505000.0,2.0,2.0,North Bay Colony Condo,1206.0,NaN,1995.0,Na

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: 1515 Flagler Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $500,000M to $6,500,000M
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, West Palm Beach, Delray Beach, Jupiter, Boca Raton, North Palm Beach, Juno Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 80 condo sales totaling $27,830,387 million from Oct. 20th to Oct. 27th. The previous week, brokers closed 78 condo sales totaling $45,900,000.

Last week’s units sold for an average of $347,880, lower than the $553,339 average price from the previous week.

In [55]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/West-Palm-Beach/200-Arkona-Ct-33401/unit-1104/home/185865981


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Riviera-Beach/3100-N-Ocean-Dr-33404/unit-1606-H/home/193264676


In [57]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/North-Palm-Beach/816-Prosperity-Farms-Rd-33408/unit-5/home/42650502


In [63]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Boca-Raton/17257-Boca-Club-Blvd-33487/unit-4/home/42299666


In [64]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/West-Palm-Beach/200-Arkona-Ct-33401/unit-1104/home/185865981


In [65]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/North-Palm-Beach/816-Prosperity-Farms-Rd-33408/unit-5/home/42650502


In [66]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-23,Condo/Co-op,200 Arkona Ct #1104,West Palm Beach,FL,33401.0,6500000.0,3.0,3.5,1515 Flagler Condo,3094.0,NaN,2023.0,NaN,2101.0,5330.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11015480,N,Y,26.699821,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Michael Costello,Compass Florida LLC (PB),Chris Deitz,Compass Florida LLC (PB),Michael Costello,Compass Florida LLC (PB),None,None,1,orange,1515 Flagler Condo at 200 Arkona Ct #1104 in W...
1,PAST SALE,2024-10-22,Condo/Co-op,3100 N Ocean Dr Unit 1606 H,West Palm Beach,FL,33404.0,1500000.0,2.0,2.5,Amrit Ocean Resort & Resi,1600.0,NaN,2023.0,NaN,938.0,2320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,F10467804,N,Y,26.787798,-80.034574,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Kari Lydon,Compass Florida LLC,None,None,Ryan Kartheiser,"Miami Condo Advisors, LLC",None,None,2,blue,Amrit Ocean Resort & Resi at 3100 N Ocean Dr U...
2,PAST SALE,2024-10-25,Condo/Co-op,500 S Ocean Blvd #604,Boca Raton,FL,33432.0,1300000.0,2.0,2.0,Chalfonte Condo,1446.0,NaN,1974.0,NaN,899.0,1569.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Beaches MLS,RX-11013796,N,Y,26.345068,-80.071816,https://www.redfin.com/FL/Boca-Raton/500-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,3,blue,Chalfonte Condo at 500 S Ocean Blvd #604 in Bo...
3,PAST SALE,2024-10-22,Condo/Co-op,2103 S Ocean Blvd Unit 3-C,Delray Beach,FL,33483.0,1275000.0,2.0,2.0,Bermuda High South,1254.0,NaN,1968.0,NaN,1017.0,1777.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2103-S-...,Beaches MLS,RX-10976860,N,Y,26.434986,-80.062616,https://www.redfin.com/FL/Delray-Beach/2103-S-...,Karen Anderson,Champagne & Parisi Real Estate,None,None,Natalia Reid,One Sotheby's International Realty,None,None,4,blue,Bermuda High South at 2103 S Ocean Blvd Unit 3...
4,PAST SALE,2024-10-25,Condo/Co-op,123 Tresana Blvd #57,Jupiter,FL,33478.0,957500.0,3.0,3.0,Jupiter Country Club Condo II,2665.0,NaN,2015.0,NaN,359.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/123-Tresana-...,Beaches MLS,RX-10994966,N,Y,26.926523,-80.157293,https://www.redfin.com/FL/Jupiter/123-Tresana-...,Mary Ellen Kavanewsky,"NV Realty Group, LLC",None,None,Anne Bosso,"Robert Bosso Rlty Ser., Inc",None,None,5,blue,Jupiter Country Club Condo II at 123 Tresana B...
5,PAST SALE,2024-10-25,Condo/Co-op,99 SE Mizner Blvd #738,Boca Raton,FL,33432.0,535000.0,1.0,1.0,Palmetto Place Condo,890.0,NaN,2003.0,NaN,601.0,868.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/99-SE-Miz...,Beaches MLS,RX-10996459,N,Y,26.348949,-80.083947,https://www.redfin.com/FL/Boca-Raton/99-SE-Miz...,Andrea Morrone,Keller Williams Realty Boca Raton,None,None,Nikki Hirsch,"United Realty Group, Inc",Stacey Swaye,"United Realty Group, Inc",6,blue,Palmetto Place Condo at 99 SE Mizner Blvd #738...
6,PAST SALE,2024-10-22,Condo/Co-op,1605 S Us Highway 1 Unit 10e,Jupiter,FL,33477.0,525000.0,2.0,2.0,Sea Lofters,1160.0,NaN,1981.0,NaN,453.0,769.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1605-S-US-Hi...,Beaches MLS,RX-11017589,N,Y,26.921071,-80.071591,https://www.redfin.com/FL/Jupiter/1605-S-US-Hi...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Bradley Ball,Frankel Ball Realty LLC,None,None,7,blue,Sea Lofters at 1605 S Us Highway 1 Unit 10e in...
7,PAST SALE,2024-10-25,Condo/Co-op,224 Bay Colony Dr N,Juno Beach,FL,33408.0,505000.0,2.0,2.0,North Bay Colony Condo,1206.0,NaN,1995.0,Na

## Time on Market Calculator

In [62]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 26) ## List (Earlier) date
date2 = datetime(2024, 10, 23) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

58


## Clean CSV for Datawrapper Chart

In [67]:
chart_df = df_top_ten

In [68]:
chart_df = chart_df.fillna(" ")

In [69]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [70]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [71]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [72]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [73]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [74]:
csv_date_string = today.strftime("%m_%d_%Y")

In [75]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [76]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-10-23,200 Arkona Ct #1104 in West Palm Beach,"$6,500,000",3,3.5,"3,094",2023,"$2,101",Beaches MLS,RX-11015480,Michael Costello Compass Florida LLC (PB) Chri...,Michael Costello Compass Florida LLC (PB)
1,2024-10-22,3100 N Ocean Dr Unit 1606 H in West Palm Beach,"$1,500,000",2,2.5,"1,600",2023,$938,Beaches MLS,F10467804,Kari Lydon Compass Florida LLC,"Ryan Kartheiser Miami Condo Advisors, LLC"
2,2024-10-25,500 S Ocean Blvd #604 in Boca Raton,"$1,300,000",2,2.0,"1,446",1974,$899,Beaches MLS,RX-11013796,Ana Londono Lang Realty/BR,Ana Londono Lang Realty/BR
3,2024-10-22,2103 S Ocean Blvd Unit 3-C in Delray Beach,"$1,275,000",2,2.0,"1,254",1968,"$1,017",Beaches MLS,RX-10976860,Karen Anderson Champagne & Parisi Real Estate,Natalia Reid One Sotheby's International Realt...
4,2024-10-25,123 Tresana Blvd #57 in Jupiter,"$957,500",3,3.0,"2,665",2015,$359,Beaches MLS,RX-10994966,"Mary Ellen Kavanewsky NV Realty Group, LLC","Anne Bosso Robert Bosso Rlty Ser., Inc"
5,2024-10-25,99 SE Mizner Blvd #738 in Boca Raton,"$535,000",1,1.0,890,2003,$601,Beaches MLS,RX-10996459,Andrea Morrone Keller Williams Realty Boca Rat...,"Nikki Hirsch United Realty Group, Inc Stacey S..."
6,2024-10-22,1605 S Us Highway 1 Unit 10e in Jupiter,"$525,000",2,2.0,"1,160",1981,$453,Beaches MLS,RX-11017589,Bradley Ball Frankel Ball Realty LLC Shannon B...,Bradley Ball Frankel Ball Realty LLC
7,2024-10-25,224 Bay Colony Dr N in Juno Beach,"$505,000",2,2.0,"1,206",1995,$419,Beaches MLS,RX-10998857,Helen McCulloch Your Home Sold Guaranteed Real...,John Rupp Premier Brokers International
8,2024-10-22,17257 Boca Club Blvd #4 in Boca Raton,"$500,000",3,2.0,"1,426",1992,$351,Beaches MLS,RX-11014286,Denise Silk RE/MAX Select Group,Irene Marshall Coldwell Banker/BR
9,2024-10-24,816 Prosperity Farms Rd #5 in North Palm Beach,"$500,000",3,3.5,"2,036",2007,$246,MARMLS,A11590465,"Courtney Smitheman Crane Reed Properties, LLC",
